## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [2]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

Notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [3]:
# local path to downloaded results
iteration = '_08' # <----- options: 'base', '_01'~'_06'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}/baseline' 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_08/baseline


In [4]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_maximum_daily_use_overlap_kw,qoi_report.average_minimum_daily_use_cooling_kw,qoi_report.average_minimum_daily_use_heating_kw,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw
0,1,15,2020-11-09 20:21:38,2020-11-09 20:23:02,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,8.916468,2.186094,1.733352,1.310927,13.818182,17.909091,13.896401,12.967543,3849.0,16.448333
1,2,12,2020-11-09 18:28:58,2020-11-09 18:31:40,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,27.426483,7.751364,6.806463,5.043686,15.000000,17.363636,47.522314,35.004335,5247.0,51.651758
2,3,37,2020-11-10 00:23:40,2020-11-10 00:25:09,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,8.241401,2.183082,1.971524,1.435452,15.727273,17.818182,15.937792,11.670555,5127.0,18.372129
3,4,24,2020-11-09 21:41:54,2020-11-09 22:07:44,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,158.398883,66.687511,18.563028,25.833641,14.727273,17.727273,339.046774,106.832002,5102.0,377.517726
4,5,13,2020-11-09 21:55:25,2020-11-09 21:56:52,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,4.355445,1.360218,1.552485,1.041577,18.000000,18.181818,9.040335,6.122418,5250.0,9.561510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,30,2020-11-09 19:27:34,2020-11-09 19:29:11,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,10.165764,1.323110,2.764280,1.353537,13.454545,18.727273,29.414365,9.057867,4525.0,33.850741
39996,39997,7,2020-11-09 21:50:30,2020-11-09 21:51:59,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,12.291725,2.086623,1.660708,1.580284,13.727273,16.000000,20.563604,17.794388,5151.0,24.928837
39997,39998,25,2020-11-09 20:44:02,2020-11-09 21:44:58,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,160.857889,62.649835,25.904439,37.161840,16.090909,15.818182,298.359059,210.906614,4028.0,388.446706
39998,39999,33,2020-11-09 21:19:26,2020-11-09 21:20:56,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,7.245476,3.252164,1.036656,1.694341,13.818182,17.363636,15.166259,7.974908,3784.0,18.065416


In [5]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'build_existing_model.hvac_system_cooling_type'
else:
    cooling_col = 'build_existing_model.hvac_cooling_type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'build_existing_model.hvac_system_heating_natural_gas'
else:
    heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')


selected


### Check housing charateristics distributions

In [6]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_08/baseline - RESULT SUMMARY:

  * 39970 / 40000 samples ran successfully, 30 failed, efficacy: 99.9% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1            0.542182
2            0.267200
3            0.190618
N_failed    30.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry      0.699299
WoodStud     0.300701
N_failed    30.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59      0.235026
1960-79      0.255041
1980-99      0.137928
2000-09      0.076132
2010s        0.011684
<1940        0.284188
N_failed    30.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC     0.616938
Heat Pump      0.025619
None           0.052389
Room AC        0.305054
N_failed      30.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP        

### Add additional metrics to ResStock results

In [21]:
# (1) add sqft and eui
sqft_map = {'0-499': 328,
            '500-749': 633,
            '750-999': 885,
            '1000-1499': 1220,
            '1500-1999': 1690,
            '2000-2499': 2176,
            '2500-2999': 2663,
            '3000-3999': 3301,
            '4000+': 8194}

res['sqft'] = res['build_existing_model.geometry_floor_area'].map(sqft_map)
res['gas_eui'] = res['simulation_output_report.total_site_natural_gas_therm'].divide(res['sqft'])
res['elec_eui'] = res['simulation_output_report.total_site_electricity_kwh'].divide(res['sqft'])


# (2) aggregate heating and cooling end uses in ResStock results
res['total_gas_heating'] = res['simulation_output_report.natural_gas_central_system_heating_therm'] +\
    res['simulation_output_report.natural_gas_heating_therm']

res['total_elec_heating'] = res['simulation_output_report.electricity_central_system_heating_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_heating_kwh'] +\
    res['simulation_output_report.electricity_fans_heating_kwh'] +\
    res['simulation_output_report.electricity_heating_kwh'] +\
    res['simulation_output_report.electricity_pumps_heating_kwh']

res['total_elec_cooling'] = res['simulation_output_report.electricity_central_system_cooling_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_cooling_kwh'] +\
    res['simulation_output_report.electricity_cooling_kwh'] +\
    res['simulation_output_report.electricity_fans_cooling_kwh'] +\
    res['simulation_output_report.electricity_pumps_cooling_kwh']

## EE prototype tags
res['vintage_ee'] = 'post-1978'
res.loc[res['build_existing_model.vintage_acs'].isin(['1940-59','1960-79']),'vintage_ee'] = '1942-1978'
res.loc[res['build_existing_model.vintage_acs']=='<1940','vintage_ee'] = 'pre-1942'

res['stories'] = '2 or more stories'
res.loc[res['build_existing_model.geometry_stories']=='1','stories'] = 'less than 2 stories'

print('new cols added to "res"')

new cols added to "res"


In [92]:
### get sqft distr by building type and vintage
res_sqft_distr = res.groupby(['build_existing_model.geometry_building_type_recs', 'build_existing_model.vintage_acs',
    'build_existing_model.geometry_floor_area'])['building_id'].count().to_frame()

# normalize
res_sqft_distr = res_sqft_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)

# export
res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_floor_area.csv'), index=True)
print('sqft distr by vintage & bldg type exported')


sqft distr by vintage & bldg type exported


In [23]:
### get foundation type distr by building type, vintage

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['build_existing_model.vintage_acs', 'build_existing_model.geometry_stories',
    'build_existing_model.geometry_foundation_type'])['building_id'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr
# export
#res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_foundation.csv'), index=True)
#print('foundation type distr by vintage & bldg type exported')


build_existing_model.geometry_foundation_type                              Crawl  \
build_existing_model.vintage_acs build_existing_model.geometry_stories             
1940-59                          1                                      0.078811   
                                 2                                      0.067132   
                                 3                                           NaN   
1960-79                          1                                      0.085336   
                                 2                                      0.083225   
                                 3                                      0.038462   
1980-99                          1                                      0.064270   
                                 2                                      0.089572   
                                 3                                      0.035714   
2000-09                          1                                      0.097484   
                                 2                                      0.090692   
                                 3                                           NaN   
2010s                            1                                      0.088889   
                                 2                                      0.081967   
                                 3                                           NaN   
<1940                            1                                      0.080157   
                                 2                                      0.080997   
                                 3                                      0.100000   

build_existing_model.geometry_foundation_type                           Heated Basement  \
build_existing_model.vintage_acs build_existing_model.geometry_stories                    
1940-59                          1                                             0.588272   
                                 2                                             0.598954   
                                 3                                             0.645161   
1960-79                          1                                             0.591273   
                                 2                                             0.563069   
                                 3                                             0.538462   
1980-99                          1                                             0.627451   
                                 2                                             0.588235   
                                 3                                             0.571429   
2000-09                          1                                             0.619497   
                                 2                                             0.565632   
                                 3                                             0.777778   
2010s                            1                                             0.600000   
                                 2                                             0.491803   
                                 3                                             0.500000   
<1940                            1                                             0.589126   
                                 2                                             0.583074   
                                 3                                             0.616667   

build_existing_model.geometry_foundation_type                               Slab  \
build_existing_model.vintage_acs build_existing_model.geometry_stories             
1940-59                          1                                      0.129549   
                                 2                                      0.134263   
                                 3                                      0.161290   
1960-79                          1                                      0.1209

In [24]:
### get foundation type distr by building type, vintage (EE prototype)

res_found_distr = res[res['build_existing_model.geometry_building_type_recs'] =="Single-Family Detached"].groupby(
    ['vintage_ee', 'stories',
    'build_existing_model.geometry_foundation_type'])['building_id'].count().to_frame()

# normalize
res_found_distr = res_found_distr.groupby(level = [0,1])['building_id'].apply(lambda x: x/x.sum()).unstack(level=2)
res_found_distr
# export
#res_sqft_distr.to_csv(os.path.join(result_path, iteration_path, 'building_type_vintage_foundation.csv'), index=True)
#print('foundation type distr by vintage & bldg type exported')


build_existing_model.geometry_foundation_type     Crawl  Heated Basement  \
vintage_ee stories                                                         
1942-1978  2 or more stories                   0.071536         0.584292   
           less than 2 stories                 0.081613         0.589561   
post-1978  2 or more stories                   0.087609         0.576953   
           less than 2 stories                 0.073380         0.624512   
pre-1942   2 or more stories                   0.081571         0.584089   
           less than 2 stories                 0.080157         0.589126   

build_existing_model.geometry_foundation_type      Slab  Unheated Basement  
vintage_ee stories                                                          
1942-1978  2 or more stories                   0.143572           0.200600  
           less than 2 stories                 0.125860           0.202966  
post-1978  2 or more stories                   0.121547           0.213891  
           less than 2 stories                 0.126464           0.175644  
pre-1942   2 or more stories                   0.133938           0.200403  
           less than 2 stories                 0.138453           0.192265

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [93]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('prototypes loaded')


prototypes loaded


In [94]:
def get_res_by_prototype(res, row):
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    return res_group_i

Metric_map1 = {'count': 'completed_status',
           'median gas': 'simulation_output_report.total_site_natural_gas_therm',
           'median elec': 'simulation_output_report.total_site_electricity_kwh'}
Metric_map2 = {'mean gas': 'simulation_output_report.total_site_natural_gas_therm',
           'mean elec': 'simulation_output_report.total_site_electricity_kwh'} # metric: res_var

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN

### (1) get median consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map1.items():
        if metric == 'count':
            groups.iloc[i, groups.columns.get_loc('count')] = \
                len(res_group_i[res_group_i['completed_status']=="Success"])
        else:
            groups.iloc[i, groups.columns.get_loc(metric)] = \
                res_group_i[res_var].median()

groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('>> median consumption computed')

### (2) get mean consumption
for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100
print('>> mean consumption computed')


>> median consumption computed
>> mean consumption computed


In [95]:
### (3) get median & mean sqft & eui
Metric_map1 = {'median sqft': 'sqft',
               'median gas eui': 'gas_eui', 
               'median elec eui': 'elec_eui'}
Metric_map2 = {'mean sqft': 'sqft', 
               'mean gas eui': 'gas_eui', 
               'mean elec eui': 'elec_eui'}

## initialize
for metric in {**Metric_map1, **Metric_map2}:
    groups[metric] = np.NAN


for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (3.1) get median values
    for metric, res_var in Metric_map1.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].median()

    ### (3.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('>> median & mean sqft & eui computed:')

groups[['HousingGroupNo','HousingGroupName','median gas','mean gas','median gas eui', 'mean gas eui',
        'median elec','mean elec', 'median elec eui','mean elec eui']]

>> median & mean sqft & eui computed:


,HousingGroupNo,HousingGroupName,median gas,mean gas,median gas eui,mean gas eui,median elec,mean elec,median elec eui,mean elec eui
0,1,Masonry Post-1978 less than 2 stories,681.822803,741.808436,0.363225,0.391620,8720.077093,9919.290563,4.525007,5.200615
1,2,Masonry All Years Split Level,985.835215,1081.326248,0.712695,0.746892,8906.977482,10129.495481,6.254167,7.096463
2,3,Masonry Post-1978 2 or more stories,1079.162765,1173.753999,0.357352,0.387074,11092.078476,12985.187521,3.739783,4.309208
3,4,Masonry 1942-1978 Less than 2 Stories,884.360096,950.957900,0.709296,0.746776,8319.854304,9320.406580,6.579341,7.403185
4,5,Masonry 1942-1978 2 or more stories,1399.397266,1499.381570,0.880233,0.904126,9889.845854,11348.829569,6.005304,7.081431
5,6,Masonry Pre-1942 less than 2 stories,981.085573,1055.100201,0.846456,0.883354,8879.542702,9686.724222,7.327519,8.330638
6,7,Masonry Pre-1942 2 or more stories,1389.063961,1453.794193,0.904489,0.925052,9892.860391,11545.871065,6.501120,7.627119
7,8,Frame Post-1978 Less than 2 stories,709.349154,770.919867,0.382368,0.419585,8992.774418,9878.763624,4.680838,5.321260
8,9,Frame Post-1978 Split Level,911.181706,1013.855105,0.375687,0.419769,10036.163548,11521.641290,4.203160,4.801451
9,10,Frame Post-1978 2 or more stories,1129.189360,1257.538989,0.370634,0.419954,11347.931894,13169.581753,3.781645,4.280040


In [96]:
### (4) get heating and cooling end uses to "groups"
Metric_map1 = ['frac_gas_heating', 'frac_elec_heating', 'frac_elec_cooling']
Metric_map2 = {'size_heating_kbut_h': 'simulation_output_report.size_heating_system_kbtu_h',
               'size_cooling_kbut_h': 'simulation_output_report.size_cooling_system_kbtu_h'}

## initialize
for metric in Metric_map1 + list(Metric_map2.keys()):
    groups[metric] = np.NAN

for i, row in groups.iterrows():
    res_group_i = get_res_by_prototype(res, row)
    
    ### (4.1) get fractions
    groups.iloc[i, groups.columns.get_loc('frac_gas_heating')] = \
        res_group_i['total_gas_heating'].mean()/res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()

    groups.iloc[i, groups.columns.get_loc('frac_elec_heating')] = \
        res_group_i['total_elec_heating'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('frac_elec_cooling')] = \
        res_group_i['total_elec_cooling'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    ### (4.2) get mean values
    for metric, res_var in Metric_map2.items():
        groups.iloc[i, groups.columns.get_loc(metric)] = \
            res_group_i[res_var].mean()

print('computed')

computed


In [97]:
### (5) get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'build_existing_model.geometry_floor_area',
              'build_existing_model.insulation_unfinished_attic']

for i, row in groups.iterrows():
    
    res_group_i = get_res_by_prototype(res, row)
    
    Ns = len(res_group_i.query('completed_status=="Success"'))
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['building_id'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [98]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   630.0   
2                                  1416                       9661  8960.0   
3                                  1812                      18225   618.0   
4                                  1210                       8779  4885.0   
5                                  1427                      10549  1159.0   
6                                  1504                       9299   783.0   
7                                  2044                      13272   885.0   
8                                  1054                       9353   651.0   
9                                  1168                      10445  1300.0   
10                                 1422                      13495   649.0   
11                                 1176                       9213  3545.0   
12                                 1297                       9964  2102.0   
13                                 1530                      12543   840.0   
14                                 1340                       9103  1001.0   
15                                 1753                      11967  1101.0   

                                                                      \
                 median gas   median elec     mean gas     mean elec   
HousingGroupNo                                                         
1                681.822803   8720.077093   741.808436   9919.290563   
2                985.835215   8906.977482  1081.326248  10129.495481   
3               1079.162765  11092.078476  1173.753999  12985.187521   
4                884.360096   8319.854304   950.957900   9320.406580   
5               1399.397266   9889.845854  1499.381570  11348.829569   
6                981.085573   8879.542702  1055.100201   9686.724222   
7               1389.063961   9892.860391  1453.794193  11545.871065   
8                709.349154   8992.774418   770.919867   9878.763624   
9                911.181706  10036.163548  1013.855105  11521.641290   
10              1129.189360  11347.931894  1257.538989  13169.581753   
11               909.369039   8403.646327   967.233908   9302.044474   
12              1160.283364   9358.191254  1230.026113  10669.582132   
13              1371.829354   9884.364284  1448.210846  11678.662046   
14               934.711393   8715.608657  1010.797374   9786.897986   
15              1357.848114  10009.154020  1429.343068  11472.095147   

                                                 ...  \
               Gas Diff-med(%) Elec Diff-med(%)  ...   
HousingGroupNo                                   ...   
1                    39.821465        15.190847  ...   
2                    30.378869         7.804808  ...   
3                    40.443556        39

### Write output to folder

In [99]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_08/baseline
